In [ ]:
#House Price Prediction


import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#print(sys.argv)

data=pd.read_csv(r"./housepriceprediction/housing_California_Housing_Data.csv")#delimiter=",")


In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')
"""

In [ ]:
data.head()
data.tail()

In [ ]:
data.info()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.info()

In [ ]:
from sklearn.linear_model import LinearRegression



In [ ]:
X = data.drop(['median_house_value'], axis=1)
y = data['median_house_value']

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
#from sklearn.feature_selection import RFE

X=data.drop(["median_house_value"],axis=1)
Y=data["median_house_value"]

In [ ]:
print(X.shape);
X #,Y

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
stscaler = StandardScaler()

#print(X.shape)
#Xnew = X.drop(columns=['ocean_proximity']) #pd.get_dummies(data,columns=['ocean_proximity'])
#Xnew = scaler.fit_transform(X)
#print(Xnew.shape)

In [ ]:
data.corr()

In [ ]:
#X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
#X_train.shape

In [ ]:
X_ = X.drop(['ocean_proximity'],axis=1)
Xnew = scaler.fit_transform(X_)
print(Xnew.shape)

In [ ]:
d_train=X.join(Y)
#this drops all columns of  ocean proximity and appends the household value

### Dataset: features and targets

Target is Median_house_value and there are 9 features with "ocean_proximity" having different class of string values. We plot a heatmap of the covariance matrix to determine the important features.

In [ ]:
d_train.sort_values(by=['median_house_value'], ascending=False)

In [ ]:
d_train.hist(figsize=(15,12))

In [ ]:
d_train.corr()
d_train.shape

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(d_train.corr(),annot=True,cmap="YlGnBu")
#Seeing The Heat Map It Can Be Noted That, The Price Of the House Is Very Much Correlated With The Area and The Number Of Bathroom"
#The Price Of The House IS Correlated With The Following In The Decreasing Order->Area->Number Of Bathrooms->Number Of Parking->Number Of Stories->Number Of Bedrooms

In [ ]:
"""
d_train["total_rooms"]=np.log(d_train["total_rooms"]+1)
d_train["total_bedrooms"]=np.log(d_train["total_bedrooms"]+1)
d_train["population"]=np.log(d_train["population"]+1)
d_train["households"]=np.log(d_train["households"]+1)
#d_train["bedrooms"]=np.log(d_train["bedrooms"]+1)
"""

In [ ]:
#d_train["ocean_proximity"].value_counts()
#Returns a Series containing counts of unique rows in the DataFrame.
#We sssign feature Of {0,1} for each condition like semifurninished, unfurnished and furnished.
#Basically a one-hot representation

In [ ]:
#pd.get_dummies(d_train["ocean_proximity"])
#Converts categorical variable into dummy/indicator variables.


In [ ]:
#Already done, need not be repeated again if d_train.shape == (20433, 14/16)
#d_train=d_train.join(pd.get_dummies(d_train.ocean_proximity)).drop(["ocean_proximity","median_house_value"],axis=1).join(data["median_house_value"])
#d_train
print(d_train.shape)

In [ ]:
plt.figure(figsize=(15,12))
sns.scatterplot(x="longitude",y="latitude",data=d_train,hue="median_house_value",palette="coolwarm")

In [ ]:
d_train["bedroom ratio"]=d_train["total_bedrooms"]/d_train["total_rooms"]
d_train["household rooms"]=d_train["total_rooms"]/d_train["households"]
d_train
d_train.drop(["median_house_value"],axis=1).join(data["median_house_value"])


In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(d_train.corr(),annot=True,cmap="YlGnBu")

## Gradient Descent Algorithm

Having looked at the confusion/covariance matrix, we find the following features to be most important in predicting the Household value:
* Median_income
* < 1 hr Proximity to the ocean
* Total number of rooms

whereas following features have a noticeable negative correlation to the Household value:
* being Inland, i.e. far away from the ocean
* bedroom to total house-area ratio

We start with the vanilla gradient descent algorithm and check the prediction efficiency of the linear regression estimator w.r.t. the dominant features.

### Gradients of a Mean-square Linear regression estimator

$$ J = \frac{1}{2m} (\hat{y} - w \cdot X^T - b)^2 $$
$$ \partial J/\partial w = -\frac{1}{2m} X^T (\hat{y} - w \cdot X^T - b)$$
$$ \partial J/\partial b = -\frac{1}{2m} (\hat{y} - w \cdot X^T - b)$$

In [ ]:
feature = d_train.drop(['median_house_value'], axis=1)
label = d_train.median_house_value
print(feature.shape)
print(label.shape)

In [ ]:
featureT = stscaler.fit_transform(feature.values)
labelT = stscaler.fit_transform(d_train.median_house_value.values.reshape(-1,1)).flatten()

In [ ]:
print(featureT.shape)
featureT

In [ ]:
print(labelT.shape)
labelT

In [ ]:
"""import numpy as np

# Generate some random data for demonstration purposes
n, d = d_train.shape   # Number of data points, # Number of features
X = featureT  # n data points with d features
y = labelT  # n data points with 1 output variable


# Initialize weights and biases
# here the initial values of the W- and b-column vectors will be fed
pickstate = np.random.randint(0,n)
w = featureT[pickstate]
b = labelT[pickstate]
print(pickstate)
print(w.shape)
print(b)
"""

In [ ]:
#w = np.random.randn(d, 1)  # d weights for the d features
#b = np.random.randn()  # Bias

import numpy as np

# Generate some random data for demonstration purposes
n, d = d_train.shape   # Number of data points, # Number of features
X = featureT  # n data points with d features
y = labelT  # n data points with 1 output variable


# Initialize weights and biases
# here the initial values of the W- and b-column vectors will be fed
pickstate = np.random.randint(0,n)
w = featureT[pickstate]
b = labelT[pickstate]
print(pickstate)
print(w.shape)
print(b)

# Set hyperparameters


alpha = 0.1 # Learning rate
tolerance = 0.001

num_iterations = 1000  # Number of iterations
CEAS1 = np.zeros((3, num_iterations))
CEAS2 = np.zeros((3, num_iterations))



# Perform gradient descent
#def gradD(alpha,w,b):
cost_at_each_step = []

for i in range(num_iterations):
    # Compute predicted values
    y_hat = np.dot(X, w) + b

    # Compute error
    error = y_hat - y

    # Compute gradients
    dw = np.dot(X.T, error) / n
    db = np.sum(error) / n

    # Update weights and biases
    w -= alpha * dw
    b -= alpha * db

    # Compute cost for monitoring convergence
    cost = np.mean(error**2)

    cost_at_each_step.append(cost)
    #if i>=1 and (cost - cost_at_each_step[i-1] <= tolerance):
    #    print("Error tolerance touched in {} steps".format(i))
    #    break
    # Print cost every 100 iterations
    if i % 100 == 0:
        print("Iteration", i, "Cost:", cost)
    #return cost_at_each_step

# Make predictions on new data
X_new = np.random.randn(10, d-1)  # 10 new data points with d features
y_pred = np.dot(X_new, w) + b
print("Predictions:", y_pred)

"""
tol = 0.001
for ai,alph in enumerate(Alpha):
    CEAS1[ai] = gradD(alph,w,b)
"""
#print(cost_at_each_step)
zz = np.diff(cost_at_each_step)
print(np.where(np.abs(zz) <= tol))


In [ ]:
CEAS1[0] = cost_at_each_step

In [ ]:
CEAS1[1] = cost_at_each_step

In [ ]:
CEAS1[2] = cost_at_each_step

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
Alpha = np.array([0.01, 0.05, 0.1])

for k in range(3):
    plt.semilogy(np.arange(num_iterations)[:320], CEAS1[k,:320]) #, label =f'alpha = {Alpha[k]}')

#plt.legend()
plt.xlabel('iteration count')
plt.ylabel('Loss value')
plt.title('Convergence for vanilla gradient descent')
plt.show()

In [ ]:
#import numpy as np

def nesterov_accelerated_gradient_descent(X, y, learning_rate, momentum, num_iterations):
    # Initialize the weight vector
    w = np.zeros(X.shape[1])
    # Initialize the bias term
    b = 0
    # Initialize the velocity vector
    v = np.zeros(X.shape[1])
    # Initialize the cost history list
    cost_history = []
    # Loop through the specified number of iterations
    for i in range(num_iterations):
        # Update the velocity using the previous weight and momentum
        v = momentum * v - learning_rate * np.dot(X, w - momentum * v) / X.shape[0]
        # Update the weight using the velocity
        w += v
        # Make predictions using the current weight vector and bias term
        y_pred = np.dot(X, w) + b
        # Calculate the cost (mean squared error)
        cost = np.mean((y_pred - y) ** 2)
        # Append the cost to the cost history list
        cost_history.append(cost)
        # Print cost every 100 iterations
        if i % 100 == 0:
            print("Iteration", i, "Cost:", cost)
    return w, b, cost_history
nesterov_accelerated_gradient_descent(X, y, learning_rate, momentum, num_iterations)

In [ ]:
# W=np.zeros((np.shape(data_X1)[0],np.shape(data_X1)[1]))
# W
class LinearRegression:
    def __init__(self, learning_rate=0.01, num_iterations=100):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = 0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        #print('shape of y=',np.shape(y))

        #initialize weights and bias to zeros
        self.weights=np.zeros((n_features,1))
        #print(np.shape(self.weights))

   #gradient descent

        for i in range(self.num_iterations):
          #computing predictions and errors
          y_pred=np.dot(X,self.weights)+self.bias
          error=y_pred-y

          #compute gradients

          delw=(1/n_samples)*np.dot(X.T,error)
          #print(np.shape(delw))
          delb=(1/n_samples)*np.sum(error)
         # print(np.shape(delb))

          #update weights and bias

          self.weights=self.weights-self.learning_rate*delw
          self.bias=self.bias-self.learning_rate*delb

    def predict(self,X):
      y_pred=np.dot(X,self.weights)+self.bias
      return y_pred

  #Creating The Linear Regression Object
lr=LinearRegression()

#generate some example data

# X=np.array([[1,2,3],[4,5,6],[7,8,9]])
# y=np.array([4.8,8.3,11.8])

#fit the model to the data

#X1 = d_train["median_income"]
#X2 = d_train["<1H OCEAN"]
#X1.head()


lr.fit(Xf1,Yt1)

#make a prediction on new data
y_pred=lr.predict(data_Xts1)
# X_new=np.array([[2,3,4],[5,6,7]])
# y_pred=lr.predict(X_new)

print(y_pred)
# print(X)



### Convergence analysis
To add a convergence criteria to the gradient descent algorithm, we can keep track of the difference between successive predictions in each iteration. We can stop the algorithm if this difference falls below the desired tolerance limit.

In [ ]:
from sklearn.linear_model import LinearRegression

X_train,Y_train=d_train.drop(["median_house_value"],axis=1),d_train["median_house_value"]
reg=LinearRegression()
reg.fit(X_train,Y_train)

In [ ]:
d_test=X_test.join(Y_test)
#d_test=d_test.drop(["bedroom_ratio","household_rooms"],axis=1)
d_test["bedroom ratio"]=d_test["total_bedrooms"]/d_test["total_rooms"]
d_test["household rooms"]=d_test["total_rooms"]/d_test["households"]
d_test

# temp_data=d_test.drop(["median_house_value","household rooms","bedroom ratio","ISLAND","NEAR OCEAN","NEAR BAY"],axis=1)
# temp_data
# temp_data=temp_data.join(d_test["ISLAND"])
# temp_data=temp_data.join(d_test["NEAR BAY"])
# temp_data=temp_data.join(d_test["NEAR OCEAN"])
# temp_data=temp_data.join(d_test["bedroom ratio"])
# temp_data=temp_data.join(d_test["household rooms"])
# temp_data=temp_data.join(d_test["median_house_value"])

# d_test=temp_data
d_test

In [ ]:
X_test,Y_test=d_test.drop(["median_house_value"],axis=1),d_test["median_house_value"]

In [ ]:
reg.score(X_test,Y_test)

In [ ]:
"""
# Define the dataset
x = [1, 2, 3, 4, 5]
y = [2, 4, 5, 4, 5]

# Initialize the parameters
theta = 0  # the parameter, including the intercept

# Set the learning rate
alpha = 0.01

# Define the tolerance limit
tolerance = 0.01

# Define the number of iterations
num_iters = 1000

# Implement the gradient descent algorithm
diff = float('inf')  # initialize the difference between successive predictions
i = 0  # initialize the number of iterations
while diff > tolerance and i < num_iters:
    h = theta * x  # the predicted values
    error = h - y  # the errors
    grad = x * error  # the gradient
    theta -= alpha * grad  # update the parameter
    new_h = theta * x  # the new predicted values
    diff = abs(new_h - h)  # the difference between successive predictions
    h = new_h
    i += 1

# Print the final value of the parameter and the number of iterations
print('theta:', theta)
print('iterations:', i)
"""

The velocity of the parameter update is initialized as v, and use it to modify the gradient update rule. The momentum parameter controls how much of the previous velocity to include in the current update. The rest of the code is the same as before, with the convergence criteria and the number of iterations being checked in the while loop.

In [ ]:
# Define the dataset
x = [1, 2, 3, 4, 5]
y = [2, 4, 5, 4, 5]

# Initialize the parameters
theta = 0  # the parameter, including the intercept
v = 0  # the velocity of the parameter update

# Set the learning rate
alpha = 0.01
momentum = 0.9

# Define the tolerance limit
tolerance = 0.01

# Define the number of iterations
num_iters = 1000

# Implement the Nesterov accelerated gradient descent algorithm
diff = float('inf')  # initialize the difference between successive predictions
i = 0  # initialize the number of iterations
while diff > tolerance and i < num_iters:
    h = (theta - momentum * v) * x  # the predicted values
    error = h - y  # the errors
    grad = x * error  # the gradient
    v = momentum * v - alpha * grad  # update the velocity
    theta += -momentum * v + (1 + momentum) * alpha * grad  # update the parameter
    new_h = (theta - momentum * v) * x  # the new predicted values
    diff = abs(new_h - h)  # the difference between successive predictions
    h = new_h
    i += 1

# Print the final value of the parameter and the number of iterations
print('theta:', theta)
print('iterations:', i)


In [ ]:
plt.contourf(theta_mesh, cost_mesh, np.log(cost_mesh), cmap=plt.cm.viridis)
plt.colorbar()
plt.plot(*zip(*path), color='red')
plt.xlabel('theta')
plt.ylabel('cost')
plt.title('Gradient Descent Iterations')
plt.show()